In [1]:
# Import libraries
import re
import nltk
import pickle
import requests

import numpy as np
import pandas as pd
import datetime as dt 
import matplotlib.pyplot as plt

from bokeh.models import *
from bokeh.plotting import *
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
from gensim.models import Word2Vec
from bokeh.layouts import column, row
from bokeh.palettes import Category20, Viridis256
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet, brown, words

In [2]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/nicolasgandar/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
data='Data/russian-troll-tweets/IRAhandle_tweets_'
pickle_files='Pickles/'
WORD_FREQ = 'Data/wordfrea.xlsx'
plot_files='Plots/'

# Introduction

The goal of this notebook is to represent the tweet by topics (tahter than category). <br>

By looking at the most popular hastag per day, we decided to define 13 arbitrary categories that seemed relevant to characterized the tweets. A first list of words was build for each topic (~ 7 words/topic), then the `Word2Vec` model was used to exten the list. The model enables to find the words that have a large (cosine) similarity the the its word space. The topic became 10 times larger. <br>

The tweet activity (number of tweets per day) for each topic was ploted. Clear peaks of activity were observable. 
Are those peaks related to a specific event?


To figure this out, we used [Wiki Portal](https://en.wikipedia.org/wiki/Portal:Current_events) to retrived the information about the event of a particular day. An automatic event dector was build. For every spike of each topic, we tryed to match an event description scrapped from Wiki Portal.<br>
We were thus able to label some peak of the tweet activity for each category.

# Data Loading & Word2Vec model

In [4]:
#loading data build in Word2vec.ipynb
def load_df(filename=(pickle_files+'df_final.pkl')):
    with open(filename, 'rb') as d:
        df=pickle.load(d)
    print(' DafaFrame loaded <--')
    return df

In [5]:
df=load_df()
df.head()

 DafaFrame loaded <--


,publish_date,content,account_category,Clean_tweet
0,2017-10-01,"""We have a sitting Democrat US Senator on tria...",RightTroll,"[sit, democrat, us, senat, trial, corrupt, bar..."
1,2017-10-01,Marshawn Lynch arrives to game in anti-Trump s...,RightTroll,"[marshawn, lynch, arriv, game, anti-trump, shi..."
2,2017-10-01,Daughter of fallen Navy Sailor delivers powerf...,RightTroll,"[daughter, fallen, navi, sailor, deliv, power,..."
3,2017-10-01,JUST IN: President Trump dedicates Presidents ...,RightTroll,"[presid, trump, dedic, presid, cup, golf, tour..."
4,2017-10-01,"19,000 RESPECTING our National Anthem! #StandF...",RightTroll,"[respect, nation, anthem, stand, anthem, stand..."


*Word Freq* is a online dictionary (wordfrequency.info) that gives the occurency the 5000 most commun english words.

In [6]:
#load data frequency from http://www.wordfrequency.info
wordfrequency = pd.read_excel(WORD_FREQ, header=0, index_col=[0], usecols=3).dropna()
wordfrequency.Frequency = wordfrequency.Frequency/(wordfrequency.Frequency.sum())
wordfrequency.rename(columns={'\xa0\xa0\xa0Word' : 'Word', 'Part of speech' : 'PoS'}, inplace=True)
wordfrequency.Word = wordfrequency.apply(lambda row: row['Word'].replace("\xa0\xa0\xa0", ''), axis=1)

#5 most communs words
wordfrequency.head(5)

,Word,PoS,Frequency
Rank,,,
1.0,the,a,0.066825
2.0,be,v,0.038041
3.0,and,c,0.032569
4.0,of,i,0.031365
5.0,a,a,0.030759


**Word2Vec model**

In [7]:
embedding=100
#non empty clean tweet
tweets=df[df.Clean_tweet.isnull()==False]['Clean_tweet'] #take non empty clean tweet

model = Word2Vec(tweets.tolist(), min_count=1, size=embedding, seed=12)
vocab_model=model.wv.vocab
print(model)

Word2Vec(vocab=33907, size=100, alpha=0.025)


# Topics

## List

We have identify a list of topic that are frequent and were a debat at the time. We will expore their behavior in the word space hopping to find clusters :
* Black lives matter, police brutality, police violence, blacktwitter, racism, NFL Protest, Jamar Clark, Alfredo Olango 
* music, thefourhorsemen, album 
* Trump, Donald
* Hillary, HeforShe, IamWithHer, crookedHillary 
* religion
* fear, North Korea, Russia, Geopolotics
* voter fraud
* terror, terrorist, attack, chicago, shootings, baltimore, bombings, Chattanooga
* hacking, emails, DNC
* Money, scandal, Wells Fargo, Imran Awan
* election, campaign, GOP, DNC, Dem, vote, I voted, debate, primary, national convention
* alt-righ, alt-left, Charlottesville, neo nazi
* economy, deal, Nafta
* music, thefourhorsemen, album, fm, nowplaying


The list will get expanded thanks to the Word2Vec model. The words that hace a (cosine) similarity larger than a thrshold (0.6) are kept and appended to the list of topics.

### Build extended list

In [10]:
stemmer = PorterStemmer()

def get_related_words(topics, model, min_similarity=0.6):
    """Get the words that are similar in the word2vec space. 
    min_similarity determines the threshold of similarity
    RETURNS: an extendent list for each topics"""
    topic_extended=[] #new list of words
    
    for topic in topics:
        new_words=[]
       
        for idx,word_raw in enumerate(topic):
            #modifies the wors as in the preprocessing
            word=stemmer.stem(word_raw.lower())
            
            if idx == 0:
                new_words.append(word_raw)
                new_words.append(word)
            
            try: #if the word is in the vocab
                related=model.wv.most_similar(word, topn=30) 
                new_words+=[related[idx][0] for idx, _ in enumerate(related) if related[idx][1] >= min_similarity]
            
            except KeyError: #the word is not in the vocabulary (anymore)
                #new_words.remove(word_raw) #then remove this word
                continue
                
        topic_extended.append(new_words)
    
    return topic_extended


#We will create a new col of stemmed words of wiki's events to compare with tweets keywords
link_numbers=('http', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-') #
stemmer = PorterStemmer()
stop_w=[word.replace('\'','') for word in stopwords.words('english')]+ ['', '&amp', 'amp','rt'] 

def tokenize(text):
    """Split the the tweet into a list of (cleaned words)"""
    text_cleaned = ''.join(ch for ch in text if ch not in '#!"$%&\()*+,./:;<=>?@[\\]^_{|}~\'').split(' ') 
    
    words= [word.lower().encode('ascii',errors='ignore').decode() for word in text_cleaned \
            if not ( (word.startswith(link_numbers)) | (word.endswith(link_numbers)) )]
    
    words=[stemmer.stem(word) for word in words if word not in stop_w ]
        
    if len(words) > 0:
        return words
    else: #tweets that contains only links or emojiis ...
        pass


In [11]:
#the firs word of the list represents the concept of each list.
topics_raw=[['Music', 'lyrics', 'album','musician', 'nowplaying', 'soundcloud','rap', 'rnb','hip-hop', 'eminem','nas', 'horsemen', 'dj', 'wayne', 'snoop', 'youtube', 'drake'],\
        ['Movies', 'tv', 'actor', 'netflix', 'trailer', 'must-see', 'episod', 'serie'],\
        ['Trump', 'melania', 'Donald', 'trump2016','realdonaldtrump', 'anti-trump', 'maga', 'MakeAmericaGreatAgain'],\
        ['Hillary', 'hilliari',  'HeforShe', 'IamWithHer', 'crookedHillary', 'clinton','killari','neverhillari', ],\
        ['Religion', 'islam', 'christianism', 'judaism'],\
        ['Conservative', 'tcot', 'ccot', 'pjnet'],\
        ['Fear', 'Korea', 'North', 'NorthKorea', 'Russia', 'putin', 'Geopolotics', 'kim', 'jong', 'kimjongun'],\
        ['BlackLivesMatter','BlackLiveMatters', 'black', 'trayvonmartin', 'mikebrown', 'sandrabland', 'bland', 'kaepernick','policeviolence', 'brutality' , 'fuckthepolice', 'cop', \
         'racial', 'blacktwitter', 'colin', 'kaepernick','racism' , 'blm', 'hill', 'MillionManMarch', 'amerikkka', 'whitesupremacy',\
         'racisme', 'blacktwitter', 'TamirRice', 'slavery', 'garner', 'pro-black', 'wearealltrayvon'],\
        ['Voter', 'fraud', 'registration'],\
        ['MeToo', 'Weinstein', 'sexual', 'harassment', 'predator', 'feminist'],\
        ['Terrorism', 'terror' 'terrorist', 'parisattack', 'londonattack', 'shootings', 'baltimore', 'bombings', 'Chattanooga', 'prayforbrussel', 'brussel', 'massacre'],\
        ['Hacking', 'emails', 'DNC'],\
        ['Sanders', 'bernie', 'bern'],\
        ['Obama', 'barack', 'michelle'],\
        ['FakeNews', 'fakenewsmedia', 'hoax', 'debunk'],\
        ['Gouvernment', 'senator', 'mcconnel', 'gongress', 'lawmaker'],\
        ['Healthcare', 'obamacare', 'medicaid', 'aca'],\
        ['Gun', 'self-defense', 'carry', 'backgroundcheck'],\
        ['Sport', 'NBA', 'basketball', 'playoff', 'hockey', 'mvp', 'football', 'soccer', 'tournament', 'championship'],\
        ['Scandal', 'money', 'WellsFargo', 'Fargo', 'ImranAwan', 'Imran', 'Awan', 'Wasserman'],\
        ['Election', 'campaign', 'GOP','gopdebate', 'DNC', 'Dem', 'demdebate', 'vote', 'Ivoted', 'debate', 'primary', 'convention', 'america'],\
        ['Charlottesville','alt-righ', 'alt-left',  'neonazi', 'neo-nazi','kkk', 'klan', 'supremacist', 'unitetheright'], \
        ['Economy', 'deal', 'Nafta','stock', 'market','bitcoin', 'nasdaq']]

print('We have chosen {} topics ({} words):\n    {}'.format(len(topics_raw), len(sum(topics_raw,[])), '\n    '.join([topic[0] for topic in topics_raw])))

topics_treated=[]
for topic in topics_raw:
    topics_treated.append([word.lower() for word in topic])

We have chosen 23 topics (181 words):
    Music
    Movies
    Trump
    Hillary
    Religion
    Conservative
    Fear
    BlackLivesMatter
    Voter
    MeToo
    Terrorism
    Hacking
    Sanders
    Obama
    FakeNews
    Gouvernment
    Healthcare
    Gun
    Sport
    Scandal
    Election
    Charlottesville
    Economy


**Extending the list:**

In [12]:
topic_extended=get_related_words(topics_raw, model, min_similarity=0.6)

print('The extended list is {} long.'.format(len(sum(topic_extended,[]))))

The extended list is 1693 long.


**Vectorizing tweets** <br>
Each tweet with respect to the topic. If a tweet contains a word contained in the list of a certain topic, then it will have a non null composant in this topic column.

In [13]:
def tweet_topics(df, topic_extended):
    """Add a column corresponding to each topic. Fills 1 if the tweet has a word 
    related to the topic, 0 otherwise """ 
    
    df_tmp=df[df['Clean_tweet'].isnull()==False]['Clean_tweet']
    for topic in topic_extended:
        df[topic[0]]= df_tmp.apply(lambda words: 1 if len(set(words) & set(topic))> 0 else 0)
        df[topic[0]].fillna(0, inplace=True)

tweet_topics(df, topic_extended)

In [14]:
df.head(10)

,publish_date,content,account_category,Clean_tweet,Music,Movies,Trump,Hillary,Religion,Conservative,...,Obama,FakeNews,Gouvernment,Healthcare,Gun,Sport,Scandal,Election,Charlottesville,Economy
0,2017-10-01,"""We have a sitting Democrat US Senator on tria...",RightTroll,"[sit, democrat, us, senat, trial, corrupt, bar...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2017-10-01,Marshawn Lynch arrives to game in anti-Trump s...,RightTroll,"[marshawn, lynch, arriv, game, anti-trump, shi...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2017-10-01,Daughter of fallen Navy Sailor delivers powerf...,RightTroll,"[daughter, fallen, navi, sailor, deliv, power,...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-10-01,JUST IN: President Trump dedicates Presidents ...,RightTroll,"[presid, trump, dedic, presid, cup, golf, tour...",0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2017-10-01,"19,000 RESPECTING our National Anthem! #StandF...",RightTroll,"[respect, nation, anthem, stand, anthem, stand...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2017-10-01,"Dan Bongino: ""Nobody trolls liberals better th...",RightTroll,"[dan, bongino, nobodi, troll, liber, better, d...",0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-10-01,🐝🐝🐝 https://t.co/MorL3AQW0z,RightTroll,None,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2017-10-01,'@SenatorMenendez @CarmenYulinCruz Doesn't mat...,RightTroll,"[senatormenendez, matter, cnn, report, crime, ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2017-10-01,"As much as I hate promoting CNN article, here ...",RightTroll,"[much, hate, promot, cnn, articl, admit, every...",0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-10-01,After the 'genocide' remark from San Juan Mayo...,RightTroll,"[genocid, remark, san, juan, mayor, narr, chan...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Plotting - tweet activity

In [15]:
topic_plot = figure(plot_width=950, plot_height=600, x_axis_type='datetime', toolbar_location="above")
#colorplot=Category20[len(topic_extended)] #form the bokeh palettes
colorplot = ['darkolivegreen', 'darkmagenta','aqua', 'aquamarine', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkolivegreen']
start_date = df.publish_date.min()
end_date = df.publish_date.max()


for color, topic in enumerate(topic_extended): #iterates over each topic
    topic_tmp=topic[0]
    df_plot=pd.DataFrame(data=df[df[topic_tmp]==1].publish_date.value_counts().sort_index())
    
    source = ColumnDataSource(data=df_plot)
    topic_plt_tmp=topic_plot.line(x='index', y='publish_date', source=source,\
            line_width=2, alpha=0.8, legend=topic_tmp, color=colorplot[color])
    
    topic_plt_tmp.visible=False

topic_plt_tmp.visible=True

topic_plot.legend.location = 'top_left'
topic_plot.legend.click_policy='hide'
topic_plot.title.text = 'Tweeting activity according topic'

hover_tool=tools.HoverTool(
    tooltips=[
        ('Date', '@index{%b %d, %Y}'),
        ('Number of tweets','@publish_date')],

    formatters={
        'index' : 'datetime', # use 'datetime' formatter for 'date' field
        'publish_date' : 'printf',   },   # use 'printf' formatter for 'adj close' field

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    attachment='above',
    show_arrow=True,
)

topic_plot.tools.append(hover_tool)

output_file(plot_files+'Topic_noEvent.html')
save(topic_plot)

'/Users/nicolasgandar/Desktop/EPFL/ADA/Projet/ADA-Proj-18/Plots/Topic_noEvent.html'

**The plot can be seen [here](Plots/Topic_noEvent.html)**

# Web scrapping

We use Wikipedia Portal which lists every event that have happened for each day. The html template is always the same enabling us to efficiently scrap it.



We want to detect the event related to the spike of tweets:
* For each categry, a threshold is define to localize the date spikes. 
* Wikipedia Portal is scrapped at the url of the date. The text follows the same pre-process as the tweets words.
* A matching of the subsection of the web page and the word of the list is done.
* We retrieve the description of the event if it its related to the topic.

In [16]:
#Getting web page
WIKI_PORTAL = "https://en.wikipedia.org/wiki/Portal:Current_events/"

def event_scrapper(date):
    """Retrieve the information of the events that happened around a date window"""
    
    #take two days before and one day after the date
    dates = [date + dt.timedelta(day,0) for day in [-1,0]]
    #daily events are stored in df
    event_df = pd.DataFrame(columns=['Date', 'Description', 'Category', 'Link'])
    
    for date in dates:
        #strftime gives 0-padded days...
        year = date.strftime("%Y")
        month = date.strftime("%B")
        day = re.sub("^[0]", "", date.strftime("%d"))  #removing 0 at beggining of day

        #right format for wiki portal and requesting html
        url_date = year + "_" + month + "_" + day
        r = requests.get(WIKI_PORTAL + url_date)
        
        #print('Response status code: {0}\n'.format(r.status_code))
        
        page_body = r.text
        soup = BeautifulSoup(page_body, 'html.parser')

        #For every category of events (dt), we look for every events from bullet list (li)
        if soup.findAll("dt"):
            
            for category in soup.findAll("dt"):

                li = category.findNext("li")
                while li:
                    #replacing \n, and removing sources: text. e.g. (CNN). 
                    #we decided to split text using regex and keeping only text before first source
                    full_text = re.split(".\s\(", li.getText().replace('\n', '. '))
                    no_source = full_text[0]
                    new_event = pd.DataFrame({'Date': [date], \
                                              'Description': [no_source],\
                                              'Category': [category.getText()], \
                                              'Link': [li.a]})


                    event_df = event_df.append(new_event, ignore_index=True, sort=False)
                    li = li.findNextSibling("li")
            
            #event_df['Stemmed_Content']=event_df.apply(lambda row: tokenize(row['Description']), axis=1)
            
        
        else: #other template of wiki portal 
            for category in soup.findAll("p"):
                category_title=category.findNext().getText()
                li = category.findNext("li")
                while li:
  
                    full_text = re.split(".\s\(", li.getText().replace('\n', '. '))
                    no_source = full_text[0]
                    new_event = pd.DataFrame({'Date': [date], \
                                              'Description': [no_source],\
                                              'Category': [category_title], \
                                              'Link': [li.a]})


                    event_df = event_df.append(new_event, ignore_index=True, sort=False)
                    li = li.findNextSibling("li")

        
    #stemming the content of the events description
    event_df['Stemmed_Content']=event_df.apply(lambda row: tokenize(row['Description']), axis=1)

    return event_df


def event_detector(df, topic_extended):
    """Detect the event by web scrapping. Returns the dataframe of events' description
    per day, the topic and the recall of the process (number of event that match over
    the number of tweet spikes)"""
    print('Srapping ...')
    
    events_df=pd.DataFrame(columns=['Date', 'Topic', 'Event', 'Stemmed'])
    nb_dates=0
    nb_detected=0
    
    for topic in topic_extended:
        topic_tmp=topic[0]
        print('  _ '+topic_tmp)
        
        #create dataframe
        df_topic=pd.DataFrame(data=df[df[topic_tmp]==1].publish_date.value_counts().sort_index().reset_index())
        df_topic.rename(columns={'index':'Date', 'publish_date': 'Count'}, inplace=True)

        #get date  where tweet activity is above the thresold
        threshold=df_topic.Count.mean() + 2.25*df_topic.Count.std()
        dates_thr=df_topic[df_topic.Count >= threshold].index
        #less than threshold?
        dates=[df_topic.loc[date,'Date'] for date in dates_thr if df_topic.loc[date-1,'Count'] < threshold]
        
        for date in dates:
            nb_dates+=1 #counter for recall score
            event=event_scrapper(date)
            
            
            try: 
                #matching the description and the words of the topic
                matching_score=event.Stemmed_Content.apply(lambda words: len(set(words) & set(topic)))
                max_match=matching_score.max() 
                #print(max_match)
                
                if (max_match > 0) : #avoid empty results
                    nb_detected+=1#counter for recall score
                   
                    #if two descriptions have the same matching score
                    for idx_max_match in matching_score[matching_score==max_match].index:
                    
                        tmp=pd.DataFrame({'Date': [event.loc[idx_max_match, 'Date']], \
                                          'Topic': [topic_tmp], \
                                          'Event': [event.loc[idx_max_match,'Description']]})
                        events_df=events_df.append(tmp, ignore_index=True, sort=False)
                        
            
            except AttributeError: #track of potential error
                print(date)
                continue
            
            except TypeError:
                print('No content has been found during scrap')
    
    
    events_df.drop_duplicates(inplace=True)
    
    return events_df, nb_detected/nb_dates, 
    
    

In [17]:
events_df, recall= event_detector(df, topic_extended)

Srapping ...
  _ Music
  _ Movies
  _ Trump
  _ Hillary
  _ Religion
  _ Conservative
  _ Fear
  _ BlackLivesMatter
  _ Voter
  _ MeToo
  _ Terrorism
  _ Hacking
  _ Sanders
  _ Obama
  _ FakeNews
  _ Gouvernment
  _ Healthcare
  _ Gun
  _ Sport
  _ Scandal
  _ Election
  _ Charlottesville
  _ Economy


# Correlation

In [18]:
print('The event detector matched {:0.2f}% ({}) of the tweet peak'.format(100*recall, events_df.shape[0]))

The event detector matched 51.88% (188) of the tweet peak


### Plotting

In [31]:
plot_threshold=True

start_date = df.publish_date.min()
end_date = df.publish_date.max()

topic_plot = figure(plot_width=950, plot_height=600, x_axis_type='datetime',\
                    y_range=[-10,4800], toolbar_location=None)

#colorplot=Category20[len(topic_extended)] #form the bokeh palettes
colorplot = ['darkolivegreen', 'darkmagenta','aqua', 'aquamarine', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkolivegreen']


ymax=[]
legend_tmp=[]


for color, topic in enumerate(topic_extended): #iterates over each topic
    topic_tmp=topic[0]
    print(topic[0])
    #tweets
    to_plot = df[df[topic_tmp]==1].publish_date.value_counts().sort_index()
    df_plot=pd.DataFrame(data=to_plot)
    ymax.append(df_plot.publish_date.max())
    
    
    if plot_threshold:
        threshold=df_plot.publish_date.mean() + 2* df_plot.publish_date.std()
        threshold_plt_tmp=topic_plot.line(x=[start_date,end_date],\
                                      y=[threshold,threshold],\
                                      color=colorplot[color], alpha=0.5, )
        threshold_plt_tmp.visible=False
    
    source = ColumnDataSource(data=df_plot)
    topic_plt_tmp=topic_plot.line(x='index', y='publish_date', source=source,\
            line_width=2, alpha=0.9, color=colorplot[color])

    
    hover_tool=tools.HoverTool(
        tooltips=[
            ('Date', '@index{%b %d, %Y}'),
            ('Number of tweets','@publish_date')],
        formatters={
            'index' : 'datetime', 
            'publish_date' : 'printf',},   
        mode='vline',
        attachment='above',
        renderers=[topic_plt_tmp])

    topic_plot.tools.append(hover_tool)
    topic_plt_tmp.visible=False

    #events
    event_plot=events_df[events_df['Topic']==topic_tmp].copy()
    event_plot['Y']=4500
    source_event=ColumnDataSource(data=event_plot)
    event_plot_tmp=topic_plot.scatter(x='Date', y='Y', source=source_event,\
                                       fill_color=colorplot[color], color=None, size=10, alpha=0.5)
    
    hover_event=tools.HoverTool(
        tooltips=[('Event', '@Event'),\
                  ('Date', '@Date{%b %d, %Y}')],
        formatters={'Event' : 'printf', 'Date' : 'datetime' },  
        mode="vline",
        attachment='right',
        show_arrow=False,
        renderers=[event_plot_tmp])

    topic_plot.tools.append(hover_event)
    event_plot_tmp.visible=False
    
    legend_tmp.append((topic_tmp,[threshold_plt_tmp, topic_plt_tmp,  event_plot_tmp]))
    
    #displays only the first line at the begining
    if color==0:
        threshold_plt_tmp.visible=True
        event_plot_tmp.visible=True
        topic_plt_tmp.visible=True


legend = Legend(items=legend_tmp,\
               click_policy='hide')
topic_plot.legend.location = 'top_left'
topic_plot.legend.orientation = "vertical"
topic_plot.legend.click_policy='hide'
topic_plot.legend.label_text_font_size='8pt'

topic_plot.title.text = 'Tweeting activity according topic'
topic_plot.yaxis.bounds=(0,max(ymax))

#output_notebook()

topic_plot.min_border_top = 75
topic_plot.min_border_bottom = 75 

topic_plot.add_layout(legend, 'left')

Music
Movies
Trump
Hillary
Religion
Conservative
Fear
BlackLivesMatter
Voter
MeToo
Terrorism
Hacking
Sanders
Obama
FakeNews
Gouvernment
Healthcare
Gun
Sport
Scandal
Election
Charlottesville
Economy


/Users/nicolasgandar/anaconda3/envs/ada/lib/python3.6/site-packages/bokeh/models/plots.py:55: UserWarning: 
You are attemptings to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/Users/nicolasgandar/anaconda3/envs/ada/lib/python3.6/site-packages/bokeh/models/plots.py:55: UserWarning: 
You are attemptings to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/Users/nicolasgandar/anaconda3/envs/ada/lib/python3.6/site-packages/bokeh/models/plots.py:55: UserWarning: 
You are attemptings to set `plot.legend.click_policy` on a plot that has zero legends added, 

In [32]:
show(topic_plot)

In [21]:
output_file(plot_files+'Topic_Events.html')
save(topic_plot)

'/Users/nicolasgandar/Desktop/EPFL/ADA/Projet/ADA-Proj-18/Plots/Topic_Events.html'

The plot can be seen [here](Plots/Topic_Events.html)

# Tweets' distribution statisitcs

In [22]:
df['Categorical'] = np.sum(df[df.columns[5:]], axis=1)

In [23]:
df.columns

Index(['publish_date', 'content', 'account_category', 'Clean_tweet', 'Music',
       'Movies', 'Trump', 'Hillary', 'Religion', 'Conservative', 'Fear',
       'BlackLivesMatter', 'Voter', 'MeToo', 'Terrorism', 'Hacking', 'Sanders',
       'Obama', 'FakeNews', 'Gouvernment', 'Healthcare', 'Gun', 'Sport',
       'Scandal', 'Election', 'Charlottesville', 'Economy', 'Categorical'],
      dtype='object')

In [24]:
def get_category_distribution(df, topic):
    '''
    Returns the percentage of tweets by topic
    '''
    results = pd.DataFrame()
    tot = len(df)
    tot_r = len(df[df.account_category == 'RightTroll'])
    tot_l = len(df[df.account_category == 'LeftTroll'])
    
    for i in range(len(topic)):
        results.loc[df.columns[i+4], 'Percent'] = np.round(100*(df[df.columns[i+4]].sum()/tot),2)
        results.loc[df.columns[i+4], 'Percent_Right'] = np.round(100*(df[df.account_category == 'RightTroll'][df.columns[i+4]].sum()/tot_r),2)
        results.loc[df.columns[i+4], 'Percent_Left'] = np.round(100*(df[df.account_category == 'LeftTroll'][df.columns[i+4]].sum()/tot_l),2)
    
    return results

def uncategorized_vocab(tweets, min_occurence=30):
    """Gathers all the words that are more frequent than min_occurence"""
    voc_raw=[]
    for words in tweets:
        if words: #if not None
            voc_raw+=words # add to voca
    count_=Counter(voc_raw) #count frequency
    uncat_voc = pd.DataFrame.from_dict(count_, orient='index').reset_index()
    uncat_voc = uncat_voc.rename(columns={'index':'word', 0:'count'})
    uncat_relevant=pd.DataFrame()
    
    for i in range(len(uncat_voc)):
        if (uncat_voc.iloc[i]['word'] not in stop_w) & (len(uncat_voc.iloc[i]['word']) > 4):
            uncat_relevant.loc[i, 'Word'] = uncat_voc.iloc[i]['word']
            uncat_relevant.loc[i, 'Count'] = uncat_voc.iloc[i]['count']

    
    return uncat_relevant

def get_uncat_voc(tweets, min_count=30):
    """Recovers the vocabulary from pickles"""
    filename=pickle_files+'uncat_vocabulary_'+str(min_count)+'.pkl'
    
    try:
        with open(filename, 'rb') as uncat_voc:
            uncat_vocabulary=pickle.load(uncat_voc) 
        print("Vocab loaded <---")
            
    except: 
        uncat_vocabulary= uncategorized_vocab(tweets)
        
        with open(filename, 'wb') as uncat_voc:
            pickle.dump(uncat_vocabulary, uncat_voc, pickle.HIGHEST_PROTOCOL) #saving the voc
            
    return uncat_vocabulary

def save_topic_df(df, filename=pickle_files +'topic_df.plk'):
   
    with open(filename, 'wb') as df_top:
        pickle.dump(df, df_top, pickle.HIGHEST_PROTOCOL)
    print('Dateframe saved -->')
        

In [25]:
topic_proportion = get_category_distribution(df, topic_extended)
cat_tweet = (100*len(df[df.Categorical > 0])/len(df))
r_cat_tweet = (100*len(df[(df.Categorical > 0) & (df.account_category == 'RightTroll')]) / len(df[df.account_category == 'RightTroll']))
l_cat_tweet = 100*len(df[(df.Categorical > 0) & (df.account_category == 'LeftTroll')]) / len(df[df.account_category == 'LeftTroll'])
print("Categorized tweets represent: {:0.2f}% of all the tweets. However, 'RighTroll' tweets are categorized in {:0.2f}%, against {:0.2f} for 'LeftTroll'".format(cat_tweet, r_cat_tweet, l_cat_tweet))

more_cat = (100*len(df[df.Categorical > 1])/len(df))
print('\n{:0.2f}% of all the tweets appear in more than one category.'.format(more_cat))

empty_tweets = (100*len(df[df.Clean_tweet.isnull()])/len(df))
print("\n{:0.2f}% of all the tweets are 'empty'.".format(empty_tweets))

df['hash_cat'] = df.hashtags.apply(lambda x: 1 if len(x) > 0 else 0)
hash_per = (100*df.hash_cat.sum()/len(df))
hash_per_r = (100*df[df.account_category == 'RightTroll'].hash_cat.sum()/len(df[df.account_category=='RightTroll']))
hash_per_l = (100*df[df.account_category == 'LeftTroll'].hash_cat.sum()/len(df[df.account_category=='LeftTroll']))
print("\nHashtags are present in: {:0.2f}% of all the tweets, 'RightTrolls' rely more on hashtags, including them in : {:0.2f}% of their tweets, compared with {:0.2f}% for 'LeftTrolls'"
      .format(hash_per, hash_per_r, hash_per_l))

Categorized tweets represent: 49.78% of all the tweets. However, 'RighTroll' tweets are categorized in 57.84%, against 36.05 for 'LeftTroll'

24.96% of all the tweets appear in more than one category.

0.47% of all the tweets are 'empty'.


AttributeError: 'DataFrame' object has no attribute 'hashtags'

In [30]:
topic_proportion.to_csv('TopicTroll.csv')

In [26]:
topic_proportion

,Percent,Percent_Right,Percent_Left
Music,3.41,0.81,7.83
Movies,1.70,1.04,2.82
Trump,17.05,22.80,7.26
Hillary,5.11,7.31,1.36
Religion,2.69,3.71,0.96
Conservative,1.97,2.91,0.37
Fear,2.75,3.15,2.08
BlackLivesMatter,6.94,4.82,10.54
Voter,2.38,2.83,1.62
MeToo,2.41,2.16,2.84


In [27]:
ax=topic_proportion[['Percent_Right','Percent_Left']].drop('Categorical').plot.bar(stacked=True, \
                                                               figsize=(18,8), color=['tomato', 'dodgerblue'])
ax.set(ylabel='Percent of tweet per topic')
plt.show()

KeyError: "['Categorical'] not found in axis"

In [ ]:
uncat_vocabulary=get_uncat_voc(df[df.Categorical == 0].Clean_tweet, min_count=30)

In [ ]:
uncat_vocabulary.sort_values(by='Count', ascending=False)

In [ ]:
word='debunk'
try:
    print('\n'.join([str(w) for w in model.wv.most_similar(stemmer.stem(word), topn=20)]))
except KeyError:
    print('Not in vocabulary or excluded during pre-processing.')

In [ ]:
print('#-Topic-# : {} '.format('\n-----\n#-Topic-# : '.join([str(topic) for topic in topic_extended])))